In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import h5py
import math as ma
from astropy.io import fits
from astropy.time import Time
from matplotlib import pyplot as plt
from matplotlib.axis import Axis
from pyuvdata import UVData
import matplotlib as mpl

In [68]:
df_sel_rfi_fast = pd.DataFrame()
transform_tb = np.repeat(np.arange(0,14,1), 4)

for n in range(1):
    #with h5py.File('/Users/eormacstudio/Documents/GitHub/GRANDWin/data/processed/detected_outliers/thermal_only/3_5/win_z_scores_data_day_1_grid_%s_integration_8_real.h5' %(n), "r") as f:
    with h5py.File('/Users/eormacstudio/Documents/GitHub/GRANDWin/data/processed/detected_outliers/win_z_scores_data_day_1_grid_%s_integration_8_real.h5' %(n), "r") as f:
        data = f["wins_z_score"][:]
        obs_id = f["obs_id"][:]
        time_blocks = f["time_blocks"][:]

    flat_data = data.reshape(-1, data.shape[-1])

    df_temp = pd.DataFrame(flat_data, columns=["XX", "XY", "YX", "YY"])

    df_temp["obs_id"] = np.tile(obs_id, data.shape[1]*data.shape[2])
    df_temp["time_index"] = np.repeat(np.arange(data.shape[0]), data.shape[1]*data.shape[2])
    df_temp["frequency"] = np.tile(np.arange(data.shape[2]), data.shape[0]*data.shape[1])
    df_temp["antenna"] = np.tile(np.repeat(np.arange(data.shape[1]), data.shape[2]), data.shape[0])
    df_temp["obs_id"] = df_temp["time_index"].map(lambda t: obs_id[t])
    df_temp["timeblock"] = df_temp["time_index"].map(lambda t: int(time_blocks[t]))

    df_sel_rfi_fast = pd.concat([df_sel_rfi_fast, df_temp]).reset_index(drop=True)

In [72]:
df_sel_rfi_fast = df_sel_rfi_fast[df_sel_rfi_fast['obs_id'] == 1095451432].reset_index(drop=True)
df_sel_rfi_fast[((df_sel_rfi_fast["XX"] < -5) | (df_sel_rfi_fast["XX"] > 5)) | ((df_sel_rfi_fast["YY"] < -5) | (df_sel_rfi_fast["YY"] > 5))]

,XX,XY,YX,YY,obs_id,time_index,frequency,antenna,timeblock
81979,5.040204,-2.717824,-2.037306,-0.159639,1095451432,85,59,0,1
83010,0.265937,-3.062092,1.675653,-5.457711,1095451432,85,450,1,1
83109,0.783009,1.050527,-1.154489,5.202476,1095451432,85,549,1,1
83851,0.427417,-1.183179,-1.578433,5.828524,1095451432,85,11,3,1
84290,0.256293,1.503156,2.327870,12.011903,1095451432,85,450,3,1
...,...,...,...,...,...,...,...,...,...
1060465,-14.704934,-7.585510,12.257628,0.220545,1095451432,96,625,120,12
1061105,15.153590,-0.376842,-22.887131,12.217790,1095451432,96,625,121,12
1062385,-5.952757,-1.295476,-0.584857,-0.055097,1095451432,96,625,123,12
1063025,-13.511202,-3.861569,2.225810,2.891237,1095451432,96,625,124,12


In [75]:
df_sel_rfi_fast[(df_sel_rfi_fast["XX"] < -5) | (df_sel_rfi_fast["YY"] < -5)]

,XX,XY,YX,YY,obs_id,time_index,frequency,antenna,timeblock
83010,0.265937,-3.062092,1.675653,-5.457711,1095451432,85,450,1,1
102850,0.693412,-3.012814,-2.001552,-5.171268,1095451432,85,450,32,1
102900,-5.836605,-1.840808,1.345662,1.950799,1095451432,85,500,32,1
109133,2.160378,1.009106,1.680363,-5.002032,1095451432,85,333,42,1
110530,0.320852,-2.851340,0.836162,-5.798030,1095451432,85,450,44,1
...,...,...,...,...,...,...,...,...,...
1058545,1.919732,0.160559,0.776230,-6.037575,1095451432,96,625,117,12
1059185,-3.203247,-2.346941,5.955657,-7.695451,1095451432,96,625,118,12
1060465,-14.704934,-7.585510,12.257628,0.220545,1095451432,96,625,120,12
1062385,-5.952757,-1.295476,-0.584857,-0.055097,1095451432,96,625,123,12


---

#### Outliers location

In [19]:
df_out_loc = pd.DataFrame()
transform_tb = np.repeat(np.arange(0,14,1), 4)

for n in range(1):
    with h5py.File('/Users/eormacstudio/Documents/GitHub/GRANDWin/data/processed/detected_outliers/outliers_location_day_1_grid_%s_integration_8_real.h5' %(n), "r") as f:
        outliers_mask = f["outliers_mask"][:]
        obs_id = f["obs_id"][:].astype(str)
        time_blocks = f["time_blocks"][:]

    flat_data = outliers_mask.reshape(-1, outliers_mask.shape[-1])

    df_outliers = pd.DataFrame(flat_data, columns=["XX", "XY", "YX", "YY"])

    df_outliers["obs_id"] = np.tile(obs_id, outliers_mask.shape[1]*outliers_mask.shape[2])
    df_outliers["time_index"] = np.repeat(np.arange(outliers_mask.shape[0]), outliers_mask.shape[1] * outliers_mask.shape[2])
    df_outliers["frequency"] = np.tile(np.arange(outliers_mask.shape[2]), outliers_mask.shape[0] * outliers_mask.shape[1])
    df_outliers["antenna"] = np.tile(np.repeat(np.arange(outliers_mask.shape[1]), outliers_mask.shape[2]), outliers_mask.shape[0])
    df_outliers["obs_id"] = df_outliers['time_index'].map(lambda t: int(obs_id[t]))
    df_outliers["timeblock"] = df_outliers['time_index'].map(lambda t: int(time_blocks[t]))

    df_out_loc = pd.concat([df_out_loc, df_outliers]).reset_index(drop=True)

In [65]:
sel_df_outliers = df_out_loc[df_out_loc['obs_id'] == 1095451432]['frequency'].value_counts().to_frame().reset_index()
sel_df_outliers = sel_df_outliers[sel_df_outliers['count'] > 3]['frequency'].to_list()

In [66]:
df_out_loc_sel = df_out_loc[df_out_loc["obs_id"] == 1095451432].reset_index(drop=True)
df_out_loc_sel = df_out_loc_sel[df_out_loc_sel[["XX", "YY"]].any(axis=1)].reset_index(drop=True)

In [67]:
df_out_loc_sel

,XX,XY,YX,YY,obs_id,time_index,frequency,antenna,timeblock
0,True,False,False,False,1095451432,85,59,0,1
1,False,False,False,True,1095451432,85,450,1,1
2,False,False,False,True,1095451432,85,549,1,1
3,False,False,False,True,1095451432,85,11,3,1
4,False,False,False,True,1095451432,85,450,3,1
...,...,...,...,...,...,...,...,...,...
2000,True,True,True,False,1095451432,96,625,120,12
2001,True,False,True,True,1095451432,96,625,121,12
2002,True,False,False,False,1095451432,96,625,123,12
2003,True,False,False,False,1095451432,96,625,124,12


In [48]:
df_out_loc_sel[["XX", "YY"]].isna()

,XX,YY
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
1146875,False,False
1146876,False,False
1146877,False,False
1146878,False,False


,XX,XY,YX,YY,obs_id,time_index,frequency,antenna,timeblock
0,True,False,False,False,1095451432,85,59,0,1
1,False,False,False,True,1095451432,85,450,1,1
2,False,False,False,True,1095451432,85,549,1,1
3,False,False,False,True,1095451432,85,11,3,1
4,False,False,False,True,1095451432,85,450,3,1
...,...,...,...,...,...,...,...,...,...
2000,True,True,True,False,1095451432,96,625,120,12
2001,True,False,True,True,1095451432,96,625,121,12
2002,True,False,False,False,1095451432,96,625,123,12
2003,True,False,False,False,1095451432,96,625,124,12
